### Loading Dataset

In [1]:
from datasets import load_dataset
billsum_dataset = load_dataset("billsum")

C:\Users\aarus\anaconda3\envs\indicf5\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
billsum_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
})

In [3]:
train = billsum_dataset['train']
test = billsum_dataset['test']

### Model & Tokenizer

In [8]:
from transformers import LEDTokenizer, LEDForConditionalGeneration

model_name = "allenai/led-base-16384"
tokenizer = LEDTokenizer.from_pretrained(model_name)
model = LEDForConditionalGeneration.from_pretrained(model_name)

In [15]:
MAX_INPUT_LENGTH = 16384  
MAX_OUTPUT_LENGTH = 512    

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples['text'], 
        max_length=MAX_INPUT_LENGTH, 
        padding="max_length",  
        truncation=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['summary'], 
            max_length=MAX_OUTPUT_LENGTH, 
            padding="max_length", 
            truncation=True
        )

    model_inputs['labels'] = labels['input_ids']
    
    return model_inputs

print("Tokenizing datasets...")
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

print("Tokenization complete.")

Tokenizing datasets...


Map:   0%|                                                                            | 0/18949 [00:00<?, ? examples/s]C:\Users\aarus\anaconda3\envs\indicf5\Lib\site-packages\transformers\tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map:  58%|█████████████████████████████████████▏                          | 11000/18949 [05:18<03:50, 34.50 examples/s]


KeyboardInterrupt: 